## Deep Learning - Hello World!

Use the TensorFlow framework to create a deep nueral network to predict hand written digits from the MNIST dataset!

In [1]:
import tensorflow as tf

In [16]:
# Load the data

mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train, x_test = x_train.reshape(-1,28*28), x_test.reshape(-1,28*28)

In [38]:
# Create method for getting batches for training

class mini_batches:
    
    def __init__(self, x, y, size):
        self.x = x
        self.y = y
        self.size = size
        self.index = 0
    
    def next_batch(self):
        if self.index + self.size >= len(self.x):            
            batch_x = self.x[self.index:]
            batch_y = self.y[self.index:]
            self.index = 0
            return batch_x, batch_y
        
        batch_x = self.x[self.index:self.index + self.size]
        batch_y = self.y[self.index:self.index + self.size]
        self.index = self.index + self.size
        return batch_x, batch_y

In [70]:
# Build the computational graph

from tensorflow.contrib.layers import fully_connected 
tf.reset_default_graph()

# Inputs for training
X = tf.placeholder(tf.float32, shape=(None,28*28), name='X')
y = tf.placeholder(tf.int32, shape=(None), name='y')

# Nueral Network layers
with tf.name_scope('network'):
    h1 = fully_connected(X, 30, scope='h1')
    h2 = fully_connected(h1, 15, scope='h2')
    output = fully_connected(h2, 10, scope='output', activation_fn=None)

# Loss from Network
with tf.name_scope('loss'):
    x_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=output)
    loss = tf.reduce_mean(x_entropy, name='loss')

# SGD
with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(.01)
    train = optimizer.minimize(loss)
    
# Evaluation of performance
with tf.name_scope('train'):
    correct = tf.nn.in_top_k(output, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()

In [76]:
# Train the model

# Mini batches
batches = mini_batches(x_train, y_train, 1000)

# Save model
saver = tf.train.Saver()

# Log files
import os
from datetime import datetime
now = datetime.utcnow().strftime('%Y%m%d%H%M%S')
log_dir = os.path.join(os.getcwd(), 'tensorflow/logs/10-deep-learning-{}/'.format(now))
acc_summary = tf.summary.scalar('accuracy',accuracy)
writer = tf.summary.FileWriter(log_dir, tf.get_default_graph())

with tf.Session() as sess:
    init.run()
    
    # SGD Updates
    for index, batch in enumerate(range(50000)):
        batch_x, batch_y = batches.next_batch()
        sess.run(train, feed_dict={X: batch_x, y:batch_y})
        
        # Checkpoints
        if index % 1000 == 0:
            saver.save(sess, os.path.join(os.getcwd(), 'tensorflow/models/10_deep_learning.ckpt'))
            log_str = acc_summary.eval(feed_dict={X: batch_x, y:batch_y})
            writer.add_summary(log_str, index)
    
    # Save final model
    saver.save(sess, os.path.join(os.getcwd(), 'tensorflow/models/10_deep_learning.ckpt'))

We have successfully built and trained our first DNN!! The accuracy seems to be quite good as well.. nearly 98%!!

![title](tensorflow/images/10-deep-learning-graph.png)

![title](tensorflow/images/10-deep-learning-accuracy.png)